# OLS Regression Visualization

### Packages and dataset

In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, BayesianRidge
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
import dash
from dash import dcc, html, Input, Output
import plotly.graph_objects as go

In [9]:
df = pd.read_csv("../data/cleaned_data.csv")
df.head()

,date,location_key,new_persons_vaccinated,cumulative_persons_vaccinated,new_confirmed,mobility_workplaces
0,2022-05-09,US_VA,930.0,7326007.0,0.0,-13.0
1,2021-11-16,US_IA,11191.0,1934013.0,0.0,-11.0
2,2022-02-03,US_GA,0.0,6766353.0,485.0,23.0
3,2021-10-15,US_KY,2794.0,2767917.0,0.0,-21.0
4,2021-04-29,US_MN,20930.0,2621229.0,205.0,-14.0


### Visual

In [14]:
# Initialize Dash app
app = dash.Dash(__name__)
app.title = "Interactive Linear Regression with DataFrame"

# Layout
app.layout = html.Div([
    html.H1("Interactive Simple Linear Regression", style={'textAlign': 'center'}),

    html.Div([
        html.Label("Adjust Slope (β₁):"),
        dcc.Slider(
            id='slope-slider', min=0, max=6, step=0.1, value=3,
            marks={i: str(i) for i in range(0, 7)}
        ),
        html.Br(),

        html.Label("Adjust Intercept (β₀):"),
        dcc.Slider(
            id='intercept-slider', min=-5, max=10, step=0.5, value=5,
            marks={i: str(i) for i in range(-5, 11, 2)}
        ),
    ], style={'width': '80%', 'margin': 'auto'}),

    dcc.Graph(id='regression-plot', style={'height': '600px'}),
])

# Callback
@app.callback(
    Output('regression-plot', 'figure'),
    Input('slope-slider', 'value'),
    Input('intercept-slider', 'value')
)
def update_plot(slope, intercept):
    X = df["mobility_workplaces"]
    Y = df["new_confirmed"]
    Y_pred = slope * X + intercept

    # Compute Mean Squared Error
    mse = np.mean((Y - Y_pred) ** 2)

    # Create Plotly figure
    fig = go.Figure()

    # Scatter plot of actual data
    fig.add_trace(go.Scatter(
        x=X, y=Y, mode='markers',
        name='Data Points', marker=dict(size=8, color='blue', opacity=0.7)
    ))

    # Regression line
    fig.add_trace(go.Scatter(
        x=X, y=Y_pred, mode='lines',
        name=f'Line: y = {slope:.2f}x + {intercept:.2f}',
        line=dict(color='red', width=3)
    ))

    # Layout
    fig.update_layout(
        title=f"Simple Linear Regression | MSE = {mse:.2f}",
        xaxis_title="X",
        yaxis_title="Y",
        template="plotly_white",
        showlegend=True
    )
    fig.write_html("../plots/OLS_plot.html", include_plotlyjs='cdn')
    return fig

In [15]:
app.run(debug=True)